In [47]:
# from inspect import currentframe, getframeinfo

# frameinfo = getframeinfo(currentframe())

# print(frameinfo.filename, frameinfo.lineno)

In [48]:
import sys
import os
from timeit import timeit
#     print(x,y)

In [49]:
from inspect import currentframe, getframeinfo
from time import process_time
import sys
import os
import datetime

class Message:
    def __init__(self,data) -> None:
        # self.name = name
        self.data = data
        self.time = datetime.datetime.now()
        # frameinfo = getframeinfo(currentframe())
        self.file_name,self.file_line = None,None
        # self.file_name,self.file_line = frameinfo.filename, sys._getframe().f_lineno

    def get_message(self):
        print('this is a pure message, data:', self.data)

    def update_message(self,**kwgars):
        for key, value in kwgars.items():
            setattr(self, key, value)

    def get_data(self):return self.data
    def get_file_line(self):return self.file_line

    def get_file_name(self):return self.file_name

    def __str__(self) -> str:
        return \
f'{self.file_name}\n\
{self.data}\n\
this data is in line number: {self.file_line}\n\
{self.time}\n'


    def start(self):...

    def end(self):...

    def update(self):...

    def refresh(self):...

    def get_compound_data(self):
        return {'file_name':self.file_name, 'code_line':self.file_line, 'data':self.data}

class STR_Message(Message):
    def __init__(self, data, description='this is a test message') -> None:
        super().__init__(data)
        self.description = description
    def __str__(self)->str:
        return f'{self.description}\n' + super().__str__()
    def get_compound_data(self):
        m=super().get_compound_data()
        m['message']= self.description


class TIM_Message(STR_Message):
    def __init__(self, data, test_string=None) -> None:
        super().__init__(data, test_string) if test_string else super().__init__(data)
        self.create_at = process_time()
        self.accumulate = 0
        self.counter = 0
    def start(self):
        self.counter +=1
        self._start = process_time()
    def end(self, ignore_warning = False):
        """
        this is a funciton called when timer end, 
        :param bool ignore_warning: a indicator of ignorance of the warning
        :param list[int] left_bc: left boundary condition
        :param list[int] right_bc: left boundary condition
        """
        self.counter -=1
        c = self.counter
        if ignore_warning == False and c != 0:
            if c < 0:
                raise MissCall('missed_call! \n \
                                    you have called end twice, they have to show in pair \n \
                                        there is risk that timer become unprecise if this happend\
                                        if you don\'t want to see this warning, set ignore warning to True').with_traceback()
            else:
                raise MissCall('missed_call! \n \
                                    you have called end twice, they have to show in pair \n \
                                        there is risk that timer become unprecise if this happend\
                                        if you don\'t want to see this warning, set ignore warning to True').with_traceback()
        self._end = process_time()
        self.accumulate += self._end - self._start
    def update(self):
        raise NotImplementedError(message ='this method has no use in time function')
    def refresh(self):
        self.accumulate =0
    # def move(self, ignore_warning = )
    def get_compound_data(self):
        m = super().get_compound_data()
        m['time'] =self.accumulate
    def __str__(self):
        return f'time_accumulate: {self.accumulate}\n' +super().__str__()

class CTR_Message(STR_Message):
    def __init__(self, data, test_string=None) -> None:
        
        super().__init__(data, test_string) if test_string else super().__init__(data)
        self.create_at = process_time()
        self.counter = 0
        self.pause = True
    def start(self):self.pause= False
    def end(self):self.pause= True
    def update(self):self.counter +=1 if self.pause == False else ...
    def refresh(self):self.counter =0
    def get_compound_data(self):
        m= super().get_compound_data()
        m['counter'] = self.counter
    def __str__(self) -> str:
        return f'count: {self.counter}\n' + super().__str__()
        
class MissCall(Exception):
    def __init__(self, message = 'missed called'):
        self.message = message
        super().__init__(self.message)
class NoSuchMessageError(Exception):
    def __init__(self,name, message = 'no such Message named: '):
        self.message = message + name
        super().__init__(self.message)


In [50]:
x = 3
m = TIM_Message(x)
c = CTR_Message(x)
m.update_message(data=4)
m.refresh()
print(c)

count: 0
this is a test message
None
3
this data is in line number: None
2022-12-15 00:39:11.150343



In [51]:
class MessagePool():
    message_pool = {}
    def new_message(self,name,data,description,index = 's'or't'or'c'):
        """
            :param string name: name of this message
            :param string index: type of the message using currently we have 
                                index [s|t|c],
                                s: string message, contain only string message
                                t: timer message, contain a timer
                                c: counter message, contain a counter
            :param data index: that store in the message
        """
        if index == 's':
            self.message_pool[name] = STR_Message(data, description)
        elif index == 't':
            self.message_pool[name] = TIM_Message(data, description)
        elif index == 'c':
            self.message_pool[name] = CTR_Message(data, description)
        else:
            raise NoSuchMessageError(name = index, message='no such message based on index: ')

    def print_message(self, name):
        message = getattr(self, name)
        print(message)
    
    def set_printing_method(self,name,function):
        #TODO not yet implemented
        self.print_function = function

    def print_data(self, name):
        try:
            print(self.message_pool['name'].get_data())
        except:
            raise NoSuchMessageError(name = name)
            
    def get_message(self, name):
        try:
            return self.message_pool[name]
        except KeyError:
            raise NoSuchMessageError(name)

    def timer(e):
        pass




In [52]:
class Tester:
    mp = MessagePool()
x= 5
Tester.mp.new_message('x', x, 'test x', 's')


以下代码主要是为了测试，在不同环境下，通过object进行访问，和通过dictionary进行访问哪个更快

<span style = "color:red">结论：dict 更快</span>

In [55]:
setup=\
"""
class Test1:
    d = {}
    def __init__(self) -> None:
        
        d = self.d
        for i in range(1000):
            d[f'{i}'] = i
    def get(self,name):
        return self.d[name]

class Test2:    
    def __init__(self) -> None:
        for i in range(1000):
            setattr(self, f'{i}', i)
    def get(self, name):
        return getattr(self,name)

test1 = Test1()
test2 = Test2()
"""
snap1=\
"""
test1.get('500')
"""
snap2 =\
"""
test2.get('500')
"""
# print(timeit(snap1,setup=setup, number =10000000) )
# print(timeit(snap2,setup=setup, number =10000000) )

# print(timeit(snap1,setup=setup, number =100000000) )
# print(timeit(snap2,setup=setup, number =100000000) )





1.181842959000278
1.313235375000204
11.786064249999981
13.03832383300005


In [ ]:

timeit('Tester.mp.get_message("x")',setup='from __main__ import Tester', number =10000000) 
timeit('')

IndentationError: expected an indented block (<timeit-src>, line 7)

In [ ]:
# timeit('Tester.mp.get_message_test("x")',setup='from __main__ import Tester', number =100000000) 

In [ ]:
# %timeit Tester.mp.get_message('x') x